<a href="https://colab.research.google.com/github/Hem-hemanth/DataAnalyticsDatasets/blob/main/CS3033_CS6405_Data_Mining_Second_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS3033/CS6405 - Data Mining - Second Assignment

### Submission

This assignment is **due on 06/04/22 at 23:59**. You should submit a single .ipnyb file with your python code and analysis electronically via Canvas.
Please note that this assignment will account for 25 Marks of your module grade.

### Declaration

By submitting this assignment. I agree to the following:

<font color="red">“I have read and understand the UCC academic policy on plagiarism, and agree to the requirements set out thereby in relation to plagiarism and referencing. I confirm that I have referenced and acknowledged properly all sources used in the preparation of this assignment.
I declare that this assignment is entirely my own work based on my personal study. I further declare that I have not engaged the services of another to either assist me in, or complete this assignment”</font>

### Objective

The Boolean satisfiability (SAT) problem consists in determining whether a Boolean formula F is satisfiable or not. F is represented by a pair (X, C), where X is a set of Boolean variables and C is a set of clauses in Conjunctive Normal Form (CNF). Each clause is a disjunction of literals (a variable or its negation). This problem is one of the most widely studied combinatorial problems in computer science. It is the classic NP-complete problem. Over the past number of decades, a significant amount of research work has focused on solving SAT problems with both complete and incomplete solvers.

Recent advances in supervised learning have provided powerful techniques for classifying problems. In this project, we see the SAT problem as a classification problem. Given a Boolean formula (represented by a vector of features), we are asked to predict if it is satisfiable or not.

In this project, we represent SAT problems with a vector of 327 features with general information about the problem, e.g., number of variables, number of clauses, fraction of horn clauses in the problem, etc. There is no need to understand the features to be able to complete the assignment.

The dataset is available at:
https://github.com/andvise/DataAnalyticsDatasets/blob/main/dm_assignment2/sat_dataset_train.csv

This is original unpublished data.

## Data Preparation

In [98]:
import pandas as pd

df = pd.read_csv("https://github.com/andvise/DataAnalyticsDatasets/blob/6d5738101d173b97c565f143f945dedb9c42a400/dm_assignment2/sat_dataset_train.csv?raw=true")
df.head()

,c,v,clauses_vars_ratio,vars_clauses_ratio,vcg_var_mean,vcg_var_coeff,vcg_var_min,vcg_var_max,vcg_var_entropy,vcg_clause_mean,...,rwh_0_max,rwh_1_mean,rwh_1_coeff,rwh_1_min,rwh_1_max,rwh_2_mean,rwh_2_coeff,rwh_2_min,rwh_2_max,target
0,420,10,42.000000,0.023810,0.600000,0.000000,0.600000,0.600000,0.000000,0.600000,...,78750.0,0.000008,0.0,7.875000e-06,0.000008,2.385082e-21,0.0,2.385082e-21,2.385082e-21,1
1,230,20,11.500000,0.086957,0.137826,0.089281,0.117391,0.160870,2.180946,0.137826,...,6646875.0,17433.722184,1.0,2.981244e-12,34867.444369,1.727721e+04,1.0,1.358551e-53,3.455442e+04,0
2,240,16,15.000000,0.066667,0.300000,0.000000,0.300000,0.300000,0.000000,0.300000,...,500000.0,1525.878932,0.0,1.525879e+03,1525.878932,1.525879e+03,0.0,1.525879e+03,1.525879e+03,1
3,424,30,14.133333,0.070755,0.226415,0.485913,0.056604,0.452830,2.220088,0.226415,...,87500.0,0.000122,1.0,6.535723e-14,0.000245,8.218628e-07,1.0,1.499676e-61,1.643726e-06,0
4,162,19,8.526316,0.117284,0.139701,0.121821,0.111111,0.185185,1.940843,0.139701,...,5859400.0,16591.494310,1.0,6.912726e-42,33182.988621,1.665903e+04,1.0,0.000000e+00,3.331807e+04,1


In [99]:
df.dtypes

c                       int64
v                       int64
clauses_vars_ratio    float64
vars_clauses_ratio    float64
vcg_var_mean          float64
                       ...   
rwh_2_mean            float64
rwh_2_coeff           float64
rwh_2_min             float64
rwh_2_max             float64
target                  int64
Length: 328, dtype: object

In [100]:
df['target'].value_counts()

1    976
0    953
Name: target, dtype: int64

check if dataframe has infinite or missing values

In [101]:
import numpy as np
ds = df.isin([np.inf, -np.inf])
print("Infinity values",ds.sum().sum())
ds2=df.isnull().sum().sum()
print("Nan values",ds2)

Infinity values 649
Nan values 15808


print the columns containing missing values

In [102]:
count=df.isnull().sum()
for i,j in count.items():
  if(int(j)>1):
    print(i,j)

v_nd_p_weights_entropy 1929
v_nd_n_weights_entropy 1929
c_nd_p_weights_entropy 1929
c_nd_n_weights_entropy 1929
rg_node_entropy 44
rg_weights_entropy 44
big_node_entropy 362
big_weights_entropy 362
and_node_entropy 787
and_weights_entropy 787
band_node_entropy 1076
band_weights_entropy 1076
exo_node_entropy 1776
exo_weights_entropy 1776


The first four columns have missing values for all rows, so we can remove(drop) them, we can use test statistic values to replace for the columns having fewer na values, and replace the columns having higher count of na values with 0

In [103]:
mean_value1=df['rg_node_entropy'].mean()
df['rg_node_entropy']=df['rg_node_entropy'].fillna(mean_value1)

mean_value2=df['rg_weights_entropy'].mean()
df['rg_weights_entropy']=df['rg_weights_entropy'].fillna(mean_value2)

mean_value3=df['big_node_entropy'].mean()
df['big_node_entropy']=df['big_node_entropy'].fillna(mean_value3)

mean_value4=df['big_weights_entropy'].mean()
df['big_weights_entropy']=df['big_weights_entropy'].fillna(mean_value4)

mean_value5=df['and_node_entropy'].mean()
df['and_node_entropy']=df['and_node_entropy'].fillna(mean_value5)

mean_value6=df['and_weights_entropy'].mean()
df['and_weights_entropy']=df['and_weights_entropy'].fillna(mean_value6)

In [104]:
df=df.fillna(0)

check if any missing values left

In [105]:
df.isnull().sum().sum()

0

In [106]:
df.replace([np.inf,-np.inf],np.nan,inplace=True)
df=df.fillna(0)

check if any infinity values left

In [107]:
ds = df.isin([np.inf, -np.inf])
print("Infinity values",ds.sum().sum())

Infinity values 0


# Tasks

## Basic models and evaluation (5 Marks)

Using Scikit-learn, train and evaluate K-NN and decision tree classifiers using 70% of the dataset from training and 30% for testing. For this part of the project, we are not interested in optimising the parameters; we just want to get an idea of the dataset. Compare the results of both classifiers.

In [108]:
# YOUR CODE HERE
X=df.iloc[:,:327]
Y=df['target']
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, Y_train)
y_pred_knn=knn_model.predict(X_test)
from sklearn import metrics
print("Accuracy for KNN:",metrics.accuracy_score(Y_test, y_pred_knn))
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model = dt_model.fit(X_train,Y_train)
y_pred_dt=dt_model.predict(X_test)
print("Accuracy for Decision Tree:",metrics.accuracy_score(Y_test,y_pred_dt))

Accuracy for KNN: 0.7806563039723662
Accuracy for Decision Tree: 0.9620034542314335


## Robust evaluation (10 Marks)

In this section, we are interested in more rigorous techniques by implementing more sophisticated methods, for instance:
* Hold-out and cross-validation.
* Hyper-parameter tuning.
* Feature reduction.
* Feature normalisation.

Your report should provide concrete information of your reasoning; everything should be well-explained.

Do not get stressed if the things you try do not improve the accuracy. The key to geting good marks is to show that you evaluated different methods and that you correctly selected the configuration.

The accuracy for knn is low, for which we try to improve the performance 

**Step 1**: Since knn is a distance based algorithm, we have to rescale the features having different units to have same scale.

In [109]:
df.describe()

,c,v,clauses_vars_ratio,vars_clauses_ratio,vcg_var_mean,vcg_var_coeff,vcg_var_min,vcg_var_max,vcg_var_entropy,vcg_clause_mean,...,rwh_0_max,rwh_1_mean,rwh_1_coeff,rwh_1_min,rwh_1_max,rwh_2_mean,rwh_2_coeff,rwh_2_min,rwh_2_max,target
count,1929.000000,1929.000000,1929.000000,1929.000000,1929.000000,1929.000000,1929.000000,1929.000000,1929.000000,1929.000000,...,1.929000e+03,1929.000000,1929.000000,1.929000e+03,1929.000000,1929.000000,1929.000000,1.929000e+03,1929.000000,1929.000000
mean,549.087092,57.689995,11.072950,0.134343,0.111101,0.209920,0.073509,0.161222,1.758409,0.111101,...,4.391681e+06,8645.001262,0.846796,1.213078e+03,16076.924800,16377.181091,0.866337,1.284981e+03,31469.380944,0.505962
std,446.746934,50.556307,8.141268,0.084303,0.103638,0.189588,0.090287,0.156326,0.898686,0.103638,...,5.428499e+06,26216.028598,0.346169,1.003112e+04,48104.066684,43597.829247,0.327328,9.951460e+03,84954.358986,0.500094
min,1.000000,2.000000,0.500000,0.022727,0.014386,0.000000,0.002368,0.015538,0.000000,0.014386,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,222.000000,27.000000,5.333333,0.070755,0.039427,0.056449,0.024882,0.058182,1.091349,0.039427,...,4.687500e+05,0.738796,0.999602,4.724060e-71,0.975101,7.152557,1.000000,0.000000e+00,12.649950,0.000000
50%,404.000000,39.000000,8.000000,0.125000,0.079902,0.176697,0.048246,0.103053,1.866538,0.079902,...,3.125000e+06,453.017692,1.000000,5.132110e-20,893.758807,976.562500,1.000000,6.033537e-81,1712.975661,1.000000
75%,776.000000,70.000000,14.133333,0.187500,0.145299,0.282668,0.085714,0.185714,2.359619,0.145299,...,6.328125e+06,7077.674591,1.000000,3.289401e-05,14155.349182,9525.920146,1.000000,2.843545e-12,19051.840291,1.000000
max,1890.000000,226.000000,44.000000,2.000000,1.000000,1.308621,1.000000,1.000000,3.959478,1.000000,...,4.062500e+07,343561.828269,1.000000,1.736111e+05,602048.861130,386723.660084,1.000000,1.736111e+05,773447.302106,1.000000


the mean values are in different scale for some attributes, so have to normalize it.
try out different scaling methods for scaling, such as min-max, standard and robust.

In [112]:
df_model1=df.copy()
df_model2=df.copy()
df_model3=df.copy()
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
scalar1=StandardScaler()
scalar2=RobustScaler()
scalar3=MinMaxScaler()
features=df.columns[:-1]
features=[np.array(features)]
for feature in features:
  df_model1[feature]=scalar1.fit_transform(df_model1[feature])
  df_model2[feature]=scalar2.fit_transform(df_model2[feature])
  df_model3[feature]=scalar3.fit_transform(df_model3[feature])
knn_1=KNeighborsClassifier()
knn_2=KNeighborsClassifier()
knn_3=KNeighborsClassifier()
X1=df_model1.drop(columns=['target'])
X2=df_model2.drop(columns=['target'])
X3=df_model3.drop(columns=['target'])
Y1=df_model1['target']
Y2=df_model2['target']
Y3=df_model3['target']
X_train_1,X_test_1,Y_train_1,Y_test_1=train_test_split(X1,Y1,test_size=0.3,random_state=42)
X_train_2,X_test_2,Y_train_2,Y_test_2=train_test_split(X2,Y2,test_size=0.3,random_state=42)
X_train_3,X_test_3,Y_train_3,Y_test_3=train_test_split(X3,Y3,test_size=0.3,random_state=42)
knn_1.fit(X_train_1,Y_train_1)
knn_2.fit(X_train_2,Y_train_2)
knn_3.fit(X_train_3,Y_train_3)
y_pred_1=knn_1.predict(X_test_1)
y_pred_2=knn_2.predict(X_test_2)
y_pred_3=knn_3.predict(X_test_3)
print("Standard Scalar Accuracy Score:",metrics.accuracy_score(Y_test_1,y_pred_1))
print("Robust Scalar Accuracy Score:",metrics.accuracy_score(Y_test_2,y_pred_2))
print("MinMax Scalar Accuracy Score:",metrics.accuracy_score(Y_test_3,y_pred_3))

Standard Scalar Accuracy Score: 0.918825561312608
Robust Scalar Accuracy Score: 0.8946459412780656
MinMax Scalar Accuracy Score: 0.8929188255613126


**NOTE**
From the above result we can see for previous default settings, applied normalization/ standardization improves data accuracy. the best off the three is Standard scalar(z-score normalisation)

In [113]:
scalar=StandardScaler()
for feature in features:
  df[feature]=scalar.fit_transform(df[feature])
X_scaled=df.drop(columns=['target'])
Y_scaled=df.target
X_train,X_test,Y_train,Y_test=train_test_split(X_scaled,Y_scaled,test_size=0.3,random_state=42)
knn_model_scaled = KNeighborsClassifier()
knn_model_scaled.fit(X_train, Y_train)
y_pred_knn=knn_model_scaled.predict(X_test)
print("Accuracy for KNN after scaling:",metrics.accuracy_score(Y_test,y_pred_knn) )

Accuracy for KNN after scaling: 0.918825561312608


The model can further be optimized by selecting the best parameters that is ideal for it, through hyperparameter tuning.
parameters can be k-value, distance metric.

In [115]:
from sklearn.model_selection import GridSearchCV
knn=KNeighborsClassifier()
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
distance_metric=[1,2]
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=distance_metric)
g_search = GridSearchCV(knn, hyperparameters, cv=10)
best_model = g_search.fit(X_train,Y_train)
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])
y_pred_tuned = best_model.predict(X_test)
print(metrics.accuracy_score(Y_test, y_pred_tuned))

Best leaf_size: 1
Best p: 1
Best n_neighbors: 1
0.9360967184801382


After trying out different parameter values, the final hyperparameters are (leaf size:1,distance_metric:1(manhattan),k-val:1)

In [157]:
knn_model_hypertuned = KNeighborsClassifier(leaf_size=1,p=1,n_neighbors=1)
knn_model_hypertuned.fit(X_train, Y_train)
y_pred_knn_2=knn_model_hypertuned.predict(X_test)
print("Accuracy for KNN after hyperparameter update:",metrics.accuracy_score(Y_test,y_pred_knn_2) )

Accuracy for KNN after hyperparameter update: 0.9360967184801382


The next method to improve the accuracy or to optimize the model is to select the right subsets, in this dataset there are 327 features, off which some might be irrelavent, we can perform feature selection strategies to remove unnecessary features.

We use rfe(wrapper method) to select best subset of features, with stratified k fold method.

In [119]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
rfecv=RFECV(estimator=RandomForestClassifier(),step=1,scoring='accuracy',cv=StratifiedKFold(10))
rfecv.fit(X_scaled,Y_scaled)

RFECV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
      estimator=RandomForestClassifier(), scoring='accuracy')

In [124]:
rfecv.support_

array([False, False, False, False, False, False, False, False, False,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False,

In [125]:
rfecv.ranking_

array([ 94,  65,  99,  36,  30,  48,  41,  43, 127,   1,   1,  35,  22,
        24, 119,  86, 115,  71,  18,  58, 172, 263,  45, 110,   1, 155,
       109, 103, 108,  70,  60,  54,  15, 107,  69,  81, 128,  97, 262,
       167,  84,  64,  44,   1,   1,   1,   1,   1,  37,  46,  52,  49,
        31,  14,  20,   8, 200,   1,   1,  19,  17,  83, 197,  62,  29,
        32,   2,   7, 198,  53, 138,  93, 143, 169, 113, 137,  95, 116,
       295,  90, 152, 160, 154, 176, 264, 310, 314, 277, 266, 292, 250,
       234, 284, 311, 316, 105, 177, 100, 156,  85, 312,  16,  68, 153,
       181,  82, 304, 299, 309, 289, 235, 272, 256, 315, 253, 296, 255,
        12, 140, 118,   1,  63, 298,  66, 150, 170,  10,  88, 279, 302,
       303, 241, 294, 260, 239, 285, 278, 269, 245, 228, 129, 185,   3,
         9, 257, 112, 161, 196, 249,  57, 261, 288, 265, 230, 293, 282,
       280, 259, 246, 225, 268,  89, 159, 202,  78,  26, 308, 120, 122,
       173, 142, 121, 131, 307, 267,  72, 125, 254, 117, 141, 19

In [133]:
df_features = pd.DataFrame(columns = ['feature', 'support', 'ranking'])

for i in range(X_scaled.shape[1]):
    row = {'feature': i, 'support': rfecv.support_[i], 'ranking': rfecv.ranking_[i]}
    df_features = df_features.append(row, ignore_index=True)
    
df_features=df_features.sort_values(by='ranking').head(15)

In [134]:
df_features.head(15)

,feature,support,ranking
58,58,True,1
57,57,True,1
24,24,True,1
255,255,True,1
120,120,True,1
44,44,True,1
45,45,True,1
10,10,True,1
9,9,True,1
43,43,True,1


The best features selected by rfecv method is listed above. We will try to fit the model based on the reduced features.

In [135]:
feature_selected=df_features[0:13]

In [152]:
index=list(feature_selected.feature)

In [153]:
index

[58, 57, 24, 255, 120, 44, 45, 10, 9, 43, 47, 46, 66]

The best subset of features selected by rfe is listed below

In [154]:
df.iloc[:,index]

,gsat_BestSolution_CoeffVariance,gsat_BestSolution_Mean,pnv_ratio_coeff,and_node_entropy,c_nd_p_node_mean,estimate_log_number_nodes_over_vars,saps_BestSolution_Mean,vcg_clause_coeff,vcg_clause_mean,mean_depth_to_contradiction_over_vars,saps_FirstLocalMinStep_Mean,saps_BestSolution_CoeffVariance,gsat_FirstLocalMinRatio_Mean
0,-0.304652,-0.629452,-0.755748,-3.539990e-15,-0.227347,0.783811,-0.634657,-1.229132,4.718609,1.062529,-0.820933,-0.241301,-2.353908
1,-0.292539,-0.116608,-0.005702,-1.908186e-01,-0.436137,-0.684806,-0.102330,-0.191928,0.257936,-1.503097,-0.489676,-0.219956,0.402830
2,-0.304652,-0.629452,-0.755748,-3.539990e-15,-0.374407,0.783811,-0.120565,-0.914755,1.823158,1.062529,-0.779232,-0.152498,-3.152663
3,-0.304652,-0.389030,-0.755748,-3.539990e-15,-0.543475,0.781486,-0.387569,-0.927489,1.112953,1.056938,-0.783630,-0.241301,0.179250
4,-0.133555,-0.597572,-0.438524,-6.124975e-01,-0.138838,0.608050,-0.580396,0.206658,0.276032,0.453980,-0.525809,-0.017285,0.499257
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924,0.278244,-0.626326,-0.544334,-1.226035e+00,2.101116,0.611423,-0.622229,1.017396,-0.637445,0.646469,0.116421,0.254841,0.534802
1925,-0.302761,-0.388934,-0.755748,-3.539990e-15,-0.534714,0.778953,-0.387420,-0.932783,1.138778,1.052068,-0.784435,-0.237365,0.090973
1926,-0.277766,-0.340705,-0.254739,1.234344e-01,-0.088910,0.273468,-0.313590,0.289767,-0.077121,-0.321855,-0.355853,-0.192378,0.459128
1927,2.222917,-0.629283,-0.223271,-3.507007e+00,2.824464,0.774728,-0.634657,2.630303,-0.887903,1.042364,2.646895,-0.241301,0.539420


In [159]:
X_final=df.iloc[:,index]
Y_final=df.target
X_train_final,X_test_final,Y_train_final,Y_test_final=train_test_split(X_final,Y_final,test_size=0.3,random_state=42)
knn_model_final = KNeighborsClassifier(leaf_size=1,p=1,n_neighbors=1)
knn_model_final.fit(X_train_final, Y_train_final)
y_pred_knn_final=knn_model_final.predict(X_test_final)
print("Accuracy for KNN after feature selection:",metrics.accuracy_score(Y_test_final,y_pred_knn_final) )

Accuracy for KNN after feature selection: 0.9930915371329879


## New classifier (10 Marks)

Replicate the previous task for a classifier that we did not cover in class. So different than K-NN and decision trees. Briefly describe your choice.
Try to create the best model for the given dataset.
Save your best model into your github. And create a single code cell that loads it and evaluate it on the following test dataset:
https://github.com/andvise/DataAnalyticsDatasets/blob/main/dm_assignment2/sat_dataset_test.csv

This link currently contains a sample of the training set. The real test set will be released after the submission. I should be able to run the code cell independently, load all the libraries you need as well.

The classifier that we will be using is Logistic Regression, a application of Gradient Boosting for decision trees. Gradient boosting is method by which the model iteratively learn from weak learners to build strong model. Here XG Boost is used because it  computationally efficient than other algorithms.

In [ ]:
# YOUR CODE HERE
from sklearn import svm


# <font color="blue">FOR GRADING ONLY</font>

Save your best model into your github. And create a single code cell that loads it and evaluate it on the following test dataset: 
https://github.com/andvise/DataAnalyticsDatasets/blob/main/dm_assignment2/sat_dataset_test.csv

In [ ]:
from joblib import dump, load
from io import BytesIO
import requests

# INSERT YOUR MODEL'S URL
mLink = 'URL_OF_YOUR_MODEL_SAVED_IN_YOUR_GITHUB_REPOSITORY?raw=true'
mfile = BytesIO(requests.get(mLink).content)
model = load(mfile)
# YOUR CODE HERE